In [1]:
from CovMat import *
import gc
from cl_and_cov import *
from signal_to_noise import *
from fisher_FoM import *
from initialize import *

In [2]:
# Input Cosmology object with a given set of parameters
cosmo = ccl.Cosmology(
    Omega_c = Omega_m-Omega_b, Omega_b=Omega_b, h=h, sigma8=sigma8, n_s=n_s,
    transfer_function=transfer_function)

In [3]:
dndz=GalaxyDistr(zi=0.001, zf=4.0, n_eff=10.024, z0=0.13, alpha=0.78)
#n_eff is chosen from the last column of Table F1,"https://arxiv.org/abs/1809.01669" LSST Science Requirements Document
#z0 and alpha are from Figure F4, Y1 of the same document

In [4]:
arcmin_per_str=(180*60/np.pi)**2
#how many square arcminutes are there in a steradian 
num_den = arcmin_per_str*10.024

In [5]:
#get data and covariance vectors
cl = {} 
cov = {}
redshifts = {}
sliced_dndz = {}

for n in range(2,6):
    for btype in ["z", "n"]:
        cov["{0}_{1}".format(btype, n)], cl["{0}_{1}".format(btype, n)], redshifts["{0}_{1}".format(btype, n)], sliced_dndz["{0}_{1}".format(btype, n)] = getDataArray(n_bins=n, bin_type=btype, cosmo=cosmo, dndz=dndz, ell=ell, numdenPerStr = num_den, fsky=0.4)

In [6]:
#binning in l
binl=10**np.linspace(np.log10(50), np.log10(8500), 20)

In [7]:
#calculating signal to noise
stn = {}
for i in ["z", "n"]:
    for j in range(2,6):
        stn["{0}_{1}".format(i,j)] = SignalToNoise(bins_in_z=j, binl=binl, cl = cl["{0}_{1}".format(i, j)], cov = cov["{0}_{1}".format(i, j)], ell=ell)

In [8]:
fisher = {}
for i in ["z", "n"]:
    for j in range(2,6):
        fisher["{0}_{1}".format(i,j)] = fisher_matrix(covariance = cov["{0}_{1}".format(i, j)])

NameError: name 'dndz_cut' is not defined